In [1]:

# Install Gradio
!pip install gradio --quiet

import gradio as gr
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load IMDB dataset
vocab_size = 10000  # Use top 10,000 words
maxlen = 200        # Max words per review

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Build RNN model
model = Sequential([
    Embedding(vocab_size, 32, input_length=maxlen),
    SimpleRNN(32),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train the model (light training)
model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test), batch_size=64)

# Word index dictionary
word_index = imdb.get_word_index()
index_word = {v + 3: k for k, v in word_index.items()}
index_word[0] = '<PAD>'
index_word[1] = '<START>'
index_word[2] = '<UNK>'
index_word[3] = '<UNUSED>'
word_to_index = {k: (v + 3) for k, v in word_index.items()}

# Preprocessing function
def encode_review(text):
    tokens = text.lower().split()
    encoded = [1]  # <START>
    for word in tokens:
        idx = word_to_index.get(word, 2)  # 2 = <UNK>
        encoded.append(idx)
    return pad_sequences([encoded], maxlen=maxlen)

# Sentiment prediction function
def predict_sentiment(review):
    encoded = encode_review(review)
    pred = model.predict(encoded)[0][0]
    sentiment = "👍 Positive" if pred > 0.5 else "👎 Negative"
    confidence = pred if pred > 0.5 else 1 - pred
    return f"{sentiment} ({confidence * 100:.2f}%)"

# Gradio UI
demo = gr.Interface(
    fn=predict_sentiment,
    inputs=gr.Textbox(lines=5, placeholder="Enter a movie review..."),
    outputs="text",
    title="🎥 Movie Review Sentiment Analysis (RNN)",
    description="Enter a movie review and get a sentiment prediction using a Recurrent Neural Network trained on IMDB data."
)

demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.2 MB/s eta 0:00:00
17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 28s 67ms/step - accuracy: 0.5727 - loss: 0.6642 - val_accuracy: 0.7909 - val_loss: 0.4744
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 64ms/step - accuracy: 0.8378 - loss: 0.3766 - val_accuracy: 0.8108 - val_loss: 0.4189
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 25s 64ms/step - accuracy: 0.9123 - loss: 0.2319 - val_accuracy: 0.8378 - val_loss: 0.3985
1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e9b4769633b2b7f7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (htt